In [1]:
import sqlite3
import pandas as pd
import time
import random
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo

In [2]:
conn = sqlite3.connect('nba_centers_info.db')

headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nba.com/',
    'Origin': 'https://www.nba.com',
    'Connection': 'keep-alive',
}

In [3]:
def get_center_data(start, end):
    # 2. Get list of active players
    active_centers = players.get_active_players()
    
    section = active_centers[start:end]
    
    center_details = []
    
    # 3. Loop through players to get specific Position info
    # Note: 'limit' is used here so you don't hit API rate limits immediately
    for player in section:
        player_id = player['id']
        print(f"Fetching details for {player['full_name']}...")
        
        try:
            # Fetch detailed info which includes the POSITION field
            info = commonplayerinfo.CommonPlayerInfo(player_id=player_id, headers = headers, timeout = 100)
            df = info.get_data_frames()[0]
            
            # Select only the relevant columns
            desired_info = df[['PERSON_ID', 'DISPLAY_FIRST_LAST', 'POSITION', 'HEIGHT', 'TEAM_NAME']]
            center_details.append(desired_info)
            
            # API rate limit protection - sleep briefly between calls
            time.sleep(random.uniform(5,10)) 
            
        except Exception as e:
            print(f"Error fetching {player['full_name']}: {e}")

    # 4. Combine all players into one DataFrame
    if center_details:
        final_df = pd.concat(center_details, ignore_index=True)

        final_df = final_df[final_df['POSITION'] == 'Center']
        
        # 5. Save to SQL table (creates table automatically if it doesn't exist)
        final_df.to_sql('Centers', conn, if_exists='append', index=False)
        print("\nSuccess! Data saved to 'players' table in nba_players.db")
        print(final_df.head())

In [ ]:
# Wait a couple minutes between each call

get_center_data(0, 100)

In [ ]:
get_center_data(101, 200)

In [ ]:
get_center_data(201, 300)

In [ ]:
get_center_data(301, 400)

In [ ]:
get_center_data(401, 500)

In [ ]:
get_center_data(501, 600)

In [ ]:
# 2. Run a SQL query to see the data
try:
    # This pulls everything from your 'Centers' table
    df = pd.read_sql("SELECT * FROM Centers", conn)
    
    print(f"Total Centers saved so far: {len(df)}")
    print("\n--- First 10 Rows ---")
    print(df.head(10))
    
except Exception as e:
    print(f"Could not find the table. Error: {e}")

conn.close()